In [6]:
!pip install langchain_community
!pip install langchain_groq
!pip install langchain_chroma
!pip install langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.0 MB/s e

In [10]:
!pip install python-dotenv

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['HUGGINGFACE_API_KEY'] = os.getenv("HUGGINGFACE_API_KEY")



In [13]:
import pandas as pd

books=pd.read_csv('books_cleaned.csv')
books.head(2)


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...


In [14]:
books['tagged_description']

,tagged_description
0,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982 A new 'Christie for Christmas' -...
2,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897 Lewis' work on the nature of lov...
4,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...
5192,9788172235222 On A Train Journey Home To North...
5193,9788173031014 This book tells the tale of a ma...
5194,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535 This collection of the timeless ...


In [15]:
books['tagged_description'].to_csv('tagged_description.txt', sep='\n', index=False, header=False)




In [16]:

raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
# by setting the chunk_size to 0 we make sure that the chunks are seperated at the seperator rather than splitting on the chunk size
documents = text_splitter.split_documents(raw_documents)



Streaming output truncated to the last 5000 lines.


In [17]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [18]:
embeddings=HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')
db_books = Chroma.from_documents(documents, embedding=embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
query="A book to teach children about nature"
docs = db_books.similarity_search(query, k=10) # getting the top 10 results
docs


[Document(id='0295444f-66d1-411f-a9b5-209c849e6f60', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='d76aec57-e850-4ae1-abbc-bd58e50acd88', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='c119404f-5d28-4692-8438-74b870ea8052', metadata={'source': '

In [20]:

books[books['isbn13'] == int(docs[0].page_content.split()[0].strip())]



,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [21]:
def retrieve_semantic_recommendations(query:str, top_k:int=10) -> pd.DataFrame:
    recs=db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)


In [22]:
retrieve_semantic_recommendations("lion in the jungle")





,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
106,9780060256753,0060256753,"Lafcadio, the Lion Who Shot Back",Shel Silverstein,Juvenile Fiction,http://books.google.com/books/content?id=lmeJR...,"""You don't have to shoot me,"" says the young l...",1963.0,4.15,112.0,5191.0,"Lafcadio, the Lion Who Shot Back","9780060256753 ""You don't have to shoot me,"" sa..."
139,9780060541835,0060541830,Congo,Michael Crichton,Fiction,http://books.google.com/books/content?id=Gp-fL...,Armed with the latest gifts of advanced techno...,2003.0,3.58,442.0,140223.0,Congo,9780060541835 Armed with the latest gifts of a...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...
189,9780060652890,0060652896,The Screwtape Letters,C. S. Lewis,Religion,http://books.google.com/books/content?id=v1kn7...,In this humorous and perceptive exchange betwe...,2001.0,4.22,224.0,116735.0,The Screwtape Letters,9780060652890 In this humorous and perceptive ...
220,9780060765453,0060765453,The Chronicles of Narnia Movie Tie-in Edition ...,C. S. Lewis,Fiction,http://books.google.com/books/content?id=4qVxv...,When Digory and Polly try to return the wicked...,2005.0,4.26,766.0,1099.0,The Chronicles of Narnia Movie Tie-in Edition ...,9780060765453 When Digory and Polly try to ret...
221,9780060765484,0060765488,"The Lion, the Witch and the Wardrobe Movie Tie...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=u95Zv...,They open a door and enter a world Narnia ... ...,2005.0,4.21,189.0,615.0,"The Lion, the Witch and the Wardrobe Movie Tie...",9780060765484 They open a door and enter a wor...
240,9780060815929,0060815922,Voice of the Gods,Trudi Canavan,Fiction,http://books.google.com/books/content?id=pRDil...,Unable to avoid being drawn into the terrible ...,2007.0,4.08,592.0,9413.0,Voice of the Gods: Age of the Five Trilogy,9780060815929 Unable to avoid being drawn into...
256,9780060845247,0060845244,"The Lion, the Witch and the Wardrobe Read-Alou...",C. S. Lewis;Pauline Baynes,Juvenile Fiction,http://books.google.com/books/content?id=deiyG...,Four English school children enter the magic l...,2005.0,4.21,189.0,56.0,"The Lion, the Witch and the Wardrobe Read-Alou...",9780060845247 Four English school children ent...
317,9780060954833,0060954833,African Nights,Kuki Gallmann,Biography & Autobiography,http://books.google.com/books/content?id=MRgkt...,"Lyrical, beautifully written tales of life in ...",2000.0,3.80,336.0,302.0,African Nights: True Stories from the Author o...,"9780060954833 Lyrical, beautifully written tal..."
433,9780064441766,0064441768,Captain Cat,Syd Hoff,Juvenile Fiction,http://books.google.com/books/content?id=sHz4s...,"A patriotic feline, Captain Cat springs out of...",1994.0,3.66,48.0,107.0,Captain Cat,"9780064441766 A patriotic feline, Captain Cat ..."
